In [5]:
from azureml.core import Workspace, Dataset, Datastore, Experiment

In [3]:
ws=Workspace.from_config("./config")

In [6]:
ws=Workspace.from_config("./config")
az_store= Datastore.get(ws,"azure_sdk_blob01")
az_dataset=Dataset.get_by_name(ws,"Titanic_vikas")
az_default_store= ws.get_default_datastore()

In [7]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

In [8]:
myenv= Environment(name="MyEnvironment")
myenv_dep= CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-interpret','azureml-explain-model','interpret-community'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)

{
    "assetId": "azureml://locations/eastus/workspaces/3e3bfef7-2a2b-41b0-9796-e1cdf285aece/environments/MyEnvironment/versions/8",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20221010.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "MyEnvironment",
    "pyt

In [9]:
from azureml.core.compute import AmlCompute 
cluster_name= "comput1"
if cluster_name not in ws.compute_targets:
    compute_config= AmlCompute.provisioning_configuration(vm_size="",
                                                        max_nodes=2)
    cluster = AmlCompute.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
    print('cluster created successfully')
else:
    cluster= ws.compute_targets[cluster_name]
    print(cluster," compute cluster found.. using it....")

AmlCompute(workspace=Workspace.create(name='aml-msft-ml-poc1', subscription_id='a4df42c9-c1d2-44c4-b2c2-05edb4b27aad', resource_group='RG-MSFT-ML-POC-01'), name=comput1, id=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourceGroups/RG-MSFT-ML-POC-01/providers/Microsoft.MachineLearningServices/workspaces/aml-msft-ml-poc1/computes/comput1, type=AmlCompute, provisioning_state=Succeeded, location=eastus, tags={})  compute cluster found.. using it....


In [ ]:
from azureml.core.compute import ComputeTarget
compute_cluster= ComputeTarget.create(ws, cluster_name, compute_config)
compute_cluster.wait_for_completion()

In [10]:
from azureml.core.runconfig import RunConfiguration 
run_config = RunConfiguration()
run_config.target= cluster_name
run_config.environment = myenv 

In [11]:
from azureml.pipeline.steps import PythonScriptStep 
from azureml.pipeline.core import PipelineData 
input_ds= ws.datasets.get("Titanic_selected_columns")
datafolder= PipelineData('datafolder', datastore=ws.get_default_datastore())

In [12]:
from azureml.core import Experiment
experiment=Experiment(workspace=ws,name="azureml_exp1_vikas")

In [13]:
dataprep_step3= PythonScriptStep(name="Datapreparation",
                                source_directory="Training_python_scripts/",
                                script_name= "Dataprep_pipeline.py",
                                inputs=[input_ds.as_named_input('raw_data')],
                                outputs=[datafolder],
                                runconfig=run_config,
                                arguments=['--datafolder', datafolder])

In [14]:
train_step3= PythonScriptStep(name="ModelTraining",
                             source_directory="Training_python_scripts/",
                             script_name="Training_pipeline_explain.py",
                             inputs=[datafolder],
                             runconfig=run_config,
                             arguments=['--datafolder',datafolder])

In [15]:
steps= [dataprep_step3,train_step3]

In [1]:
from azureml.pipeline.core import Pipeline 
new_pipeline = Pipeline(workspace=ws, steps=steps)

c:\Users\Vikas.Yetintala\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


NameError: name 'ws' is not defined

In [17]:
#new_experiment= Experiment(workspace=ws, name="pipelineexp01")
pipeline_exp_run= experiment.submit(new_pipeline)
pipeline_exp_run.wait_for_completion(show_output=True)

Created step Datapreparation [bcd6f69e][b09e6dcb-9c43-4913-a6ef-cbead9d2c73d], (This step will run and generate new outputs)Created step ModelTraining [7fec3f77][492cce95-8a3b-462c-adb3-3ef7d9a92b21], (This step will run and generate new outputs)

Submitted PipelineRun 14648df7-02a2-45b1-9f20-bbe28ba8ac17
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/14648df7-02a2-45b1-9f20-bbe28ba8ac17?wsid=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourcegroups/RG-MSFT-ML-POC-01/workspaces/aml-msft-ml-poc1&tid=3882b70d-a91e-468c-9928-820358bfbd73
PipelineRunId: 14648df7-02a2-45b1-9f20-bbe28ba8ac17
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/14648df7-02a2-45b1-9f20-bbe28ba8ac17?wsid=/subscriptions/a4df42c9-c1d2-44c4-b2c2-05edb4b27aad/resourcegroups/RG-MSFT-ML-POC-01/workspaces/aml-msft-ml-poc1&tid=3882b70d-a91e-468c-9928-820358bfbd73
PipelineRun Status: Running


StepRunId: 611a5733-e1b3-4457-90cb-b58d70cf413b
Link to Azure Machine Learning Portal: ht

'Finished'